In [361]:
%matplotlib inline
from collections import deque
import warnings
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

In [362]:
def MakeEmpty():
    return {}

In [363]:
def IsEmpty(G):
    if len(G) == 0:
        return True
    else:
        return False

In [364]:
def IsNeighbor(G, u, v):
    return u in G[v]

In [365]:
def AdjList(G,u):
    return G[u]

In [366]:
def printG(G):
    for v in G:
        print(v, end='')
        for kid in AdjList(G,v):
            print(' -> ' + str(kid), end='')
        print()
    print('\n')

In [367]:
def transpose(G):
    G_T = MakeEmpty()
    for node in G:
        for kid in G[node]:
            if kid in G_T:
                G_T[kid].append(node)
            else:
                G_T[kid] = [node] 
    return G_T


In [368]:
def BFS(G,s):
    if s not in G:
        print(f'Vertex {s} not in graph!')
        return

    d = {}
    Q = deque()
    parent = {}

    # init
    for u in G:
        d[u] = float('inf')
        parent[u] = None
        for v in G[u]:
            d[v] = float('inf')
            parent[v] = None
            
    d[s] = 0
    Q.append(s)

    # main loop
    while len(Q)>0:
        u = Q.popleft() 
        if u in G: 
            for v in G[u]: 
                if d[v] == float('inf'):
                    d[v] = d[u] + 1
                    parent[v] = u 
                    Q.append(v) 
    return d,parent


In [369]:
def makeNotDirected(G):
# Converts a directed graph to an undirected graph
    directedG = MakeEmpty()
    for v in G:
        if v in directedG:
            for kid in G[v]:
                directedG[v].append(kid)
        else:
            directedG[v] = G[v]        
        for kid in G[v]:
            if kid in directedG:
                directedG[kid].append(v)
            else:
                directedG[kid] = [v]
    return directedG

In [370]:
def makeGfromDis(distance,G,directed):
    # Generates a graph from the list of distance
    BFS_G = MakeEmpty()
    for parent in G:
        for kid in G[parent]:
            if distance[kid] != float('inf'):
                if distance[kid] == (distance[parent] + 1):                
                    if parent in BFS_G:
                        BFS_G[parent].append(kid)
                    else:
                        BFS_G[parent] = [kid]
    if directed == False:
        BFS_G = makeNotDirected(BFS_G)

    return BFS_G

In [371]:
# Generates a graph from the list of parents
def makeG(parentList,choise):
    G = MakeEmpty()
    for kid in parentList:
        if parentList[kid] == None:
            continue
        else:
            for v in parentList[kid]:
                if v in G:
                    G[v].append(kid)
                else:
                    G[v] = [kid]
    
    ## If we send an undirected graph (choise = False), we will make sure that the returned graph is as well
    if choise == False:
        G = makeNotDirected(G)

    return G

In [372]:
def FindPathBetween2V(G,s,t):
    if s not in G:
        print(f'Vertex {s} not in graph!')
        return
    
    if t not in G:
        if [t] not in G.values():
            print(f'Vertex {t} not in graph!')
            return

# Finding the graph of routes from s to t
    matching_pairs_parets = MakeEmpty()
    dis = {}
    dis[s] = 0
    d,parents=BFS(G,s)
    G_BFS = makeGfromDis(d,G,True)
    print('!!!')
    print('distance: ', d)

    if t not in G_BFS:
        if [t] not in G_BFS.values():
            print(f'There is no route from point {s} to {t}!')
            return

    d_T,parents_T = BFS(transpose(G_BFS),t)
    G_tranpose_BFS = makeGfromDis(d_T,transpose(G_BFS),True)
    print('The directed graph G transpose after running the BFS algorithm from vertex T:')
    printG(G_tranpose_BFS)
    print(d_T)

# Finds the vertices and graphs on both graphs
    for parent in G_BFS:
        for kid in G_BFS[parent]:
            if kid in G_tranpose_BFS:
                if parent in G_tranpose_BFS[kid]:
                    if d[kid] != float('inf'):
                        if kid in matching_pairs_parets:
                            matching_pairs_parets[kid].append(parent)
                        else:
                            matching_pairs_parets[kid] = [parent]
                        dis[kid] = d[kid]
            
    path_G = makeG(matching_pairs_parets,True)
    return path_G, dis


In [373]:
def path2vUndirected(G,s,t):
    # Finds the BFS from S
    d,parents = BFS(G,s)
    print('distance: ',d)
    # Direct the edges by BFS
    directedG = makeGfromDis(d,G,True)
    print('Direct the arcs from one level to the next:\n')
    printG(directedG)

    if t not in directedG:
        if [t] not in directedG.values():
            print(f'There is no route from point {s} to {t}!')
            return

    # Finds the routs from s to T in the directed graph
    directedG_BFS,d = FindPathBetween2V(directedG,s,t)
    print('Path from S to T in directed graph:')
    printG(directedG_BFS)

    # Deletes the directions of the edes
    undirectedG = makeNotDirected(directedG_BFS)
  
    
    return undirectedG, d


In [374]:
def initialize(G,s):
    d = {}
    parent = {}
    for u in G:
        d[u] = np.inf
        parent[u] = None
    d[s] = 0
    return d,parent


In [375]:
def relax(u,v,w,d,parent):
    if d[v] > d[u] + w[u,v]:
        d[v] = d[u] + w[u,v]
        parent[v] = u
    return d,parent

In [376]:
def weightTranspose(weight):
    weightT = {}
    for (s,t) in weight:
        weightT[t,s] = weight[s,t]
    return weightT       

In [377]:
def DAG_Shortest_Path(G,s,w):
    if s not in G:
        print(f'Vertex {s} not in graph!')
        return

    #pre-Algorithm
    G_nx = nx.DiGraph(G)
    sortedG = list(nx.topological_sort(G_nx))

    print('Order after topological sort:')
    print(sortedG)

    d, parent = initialize(G_nx,s)
    print('Graph after initialize:')
    print('Distance:', d)
    #Main loop
    for u in sortedG:
        for v in AdjList(G_nx,u):
            d,parent = relax(u,v,w,d,parent)
    return d,parent

In [378]:
def makeGfromDis_wighted(distance,G,weight):
    # Generates a graph from the list of distance
    G_new= MakeEmpty()
    for parent in G:
        for kid in G[parent]:
            if distance[kid] != float('inf'):
                if distance[kid] == (distance[parent] + weight[parent,kid]):                
                    if parent in G_new:
                        G_new[parent].append(kid)
                    else:
                        G_new[parent] = [kid]

    return G_new

In [379]:
def FindPathBetween2V_wieghted(G,s,t,weight):
    if s not in G:
        print('Vertex not in graph!')
        return
    
    if t not in G:
        if [t] not in G.values():
            print(f'Vertex {t} not in graph!')
            return

# Finding the graph of routes from s to t
    matching_pairs_parets = MakeEmpty()
    dis = {}
    dis[s] = 0
    d,parents=DAG_Shortest_Path(G,s,weight)
    G_BFS = makeGfromDis_wighted(d,G,weight)
    print('!!!!!')
    print('Graph G after running DAG - shortest - path from vertex S:')
    printG(G_BFS)
    print('Distance: ',d)

    if t not in G_BFS:
        if [t] not in G_BFS.values():
            print(f'There is no route from point {s} to {t}!')
            return
    print('The graph G transpose: ')
    printG(transpose(G_BFS))
    print('Weieght transpose: ', weightTranspose(weight))
    d_T,parents_T = DAG_Shortest_Path(transpose(G_BFS),t,weightTranspose(weight))
    G_tranpose_BFS = makeGfromDis_wighted(d_T,transpose(G_BFS),weightTranspose(weight))
    print('Graph G transpose after running DAG - shortest - path from vertex T:')
    printG(G_tranpose_BFS)
    print('Distance: ',d_T)


# Finds the vertices and graphs on both graphs
    for parent in G_BFS:
        for kid in G_BFS[parent]:
            if kid in G_tranpose_BFS:
                if parent in G_tranpose_BFS[kid]:
                    if d[kid] != float('inf'):
                        if kid in matching_pairs_parets:
                            matching_pairs_parets[kid].append(parent)
                        else:
                            matching_pairs_parets[kid] = [parent]
                        dis[kid] = d[kid]

    path_G = makeG(matching_pairs_parets,True)
    return path_G, dis

In [380]:
#Undirected graph
G_notDirected = { 'S': ['A', 'B','D'],
    'A': ['S', 'B','C'],
    'B': ['A', 'S', 'D','F','G'],
    'C': ['A','G','E'],
    'D': ['S','B','F'],
    'E': ['C','H'],
    'F': ['B','D','J'],
    'G': ['B','J','I','H','C'],
    'H': ['E','G','T'],
    'I': ['J','G','T'],
    'J': ['F','G','I'],
    'T': ['I','H'],
    'M': ['N','O'],
    'N': ['M'],
    'O': ['K','M'],
    'K': ['O']
    }

#Directed gragh
G_directed = {'S': ['A','B','K'],
     'A': ['B','C'],
     'B': ['G','D'],
     'D': ['F'],
     'C': ['G'],
     'E': ['C','H'],
     'H': ['G','T'],
     'G': ['I'],
     'F': ['J'],
     'J': ['I'],
     'I': ['T'],
     'K': ['L'],
     'L': ['M'],
     'M': ['T'],
     }

G_weighted = {
    'S': ['D','A','L'],
    'A': ['B'],
    'B': ['G', 'H'],
    'C': ['T'],
    'D': ['G'],
    'G': ['C'],
    'H': ['T'],
    'K': ['T'],
    'L': ['B'],
    'T': []
}

weight ={('S','A') : 0,
         ('S','L') : 0,
         ('L','B') : 1,
        ('S', 'D'): 10,
        ('A', 'B'): 1,
        ('B', 'G'): -3,
        ('B', 'H'): -1,
        ('C', 'T'): -9,
        ('D', 'G'): 3,
        ('G', 'C'): -4,
        ('H', 'T'): -15,
        ('K','T'):-3
}


# BFS for directed graph:

In [381]:
print('The directed graph G :')
printG(G_directed)

print('The directed graph G after running the BFS algorithm from vertex S :')
dis, par = BFS(G_directed,'S')
printG(makeGfromDis(dis,G_directed,True))
print('distance :',dis)
print()

print('Attempting to run BFS on a vertex that does not exist in the graph: ')
BFS(G_directed,'P')

The directed graph G :
S -> A -> B -> K
A -> B -> C
B -> G -> D
D -> F
C -> G
E -> C -> H
H -> G -> T
G -> I
F -> J
J -> I
I -> T
K -> L
L -> M
M -> T


The directed graph G after running the BFS algorithm from vertex S :
S -> A -> B -> K
A -> C
B -> G -> D
D -> F
G -> I
F -> J
I -> T
K -> L
L -> M
M -> T


distance : {'S': 0, 'A': 1, 'B': 1, 'K': 1, 'C': 2, 'G': 2, 'D': 2, 'F': 3, 'E': inf, 'H': inf, 'T': 4, 'I': 3, 'J': 4, 'L': 2, 'M': 3}

Attempting to run BFS on a vertex that does not exist in the graph: 
Vertex P not in graph!


# The Graph of the shortest paths from S to T in a directed graph:

In [382]:
print('The directed graph G :')
printG(G_directed)
print()

G,distance = FindPathBetween2V(G_directed,'S','T')
print('The graph of the shortest routes from S to T in a directed graph:')
printG(G)
print('distance:',distance)

print()
print('Trying to find the shortest route between two vertices even though there are no routes between them: ')
FindPathBetween2V(G_directed,'K','B')

The directed graph G :
S -> A -> B -> K
A -> B -> C
B -> G -> D
D -> F
C -> G
E -> C -> H
H -> G -> T
G -> I
F -> J
J -> I
I -> T
K -> L
L -> M
M -> T



!!!
distance:  {'S': 0, 'A': 1, 'B': 1, 'K': 1, 'C': 2, 'G': 2, 'D': 2, 'F': 3, 'E': inf, 'H': inf, 'T': 4, 'I': 3, 'J': 4, 'L': 2, 'M': 3}
The directed graph G transpose after running the BFS algorithm from vertex T:
B -> S
K -> S
G -> B
I -> G
T -> I -> M
L -> K
M -> L


{'A': inf, 'S': 4, 'B': 3, 'K': 3, 'C': inf, 'G': 2, 'D': inf, 'F': inf, 'I': 1, 'J': inf, 'T': 0, 'M': 1, 'L': 2}
The graph of the shortest routes from S to T in a directed graph:
S -> B -> K
B -> G
G -> I
I -> T
M -> T
K -> L
L -> M


distance: {'S': 0, 'B': 1, 'K': 1, 'G': 2, 'I': 3, 'T': 4, 'L': 2, 'M': 3}

Trying to find the shortest route between two vertices even though there are no routes between them: 
!!!
distance:  {'S': inf, 'A': inf, 'B': inf, 'K': 0, 'C': inf, 'G': inf, 'D': inf, 'F': inf, 'E': inf, 'H': inf, 'T': 3, 'I': inf, 'J': inf, 'L': 1, 'M': 2}

# BFS for undirected graph:

In [383]:
print('The undirected graph G :')
printG(G_notDirected)

dis, par = BFS(G_notDirected,'S')

print('The undirected graph G after running the BFS algorithm :')
printG(makeGfromDis(dis,G_notDirected,False))
print('distance :', dis)
print()

print('Attempting to run BFS on a vertex that does not exist in the graph: ')
BFS(G_notDirected,'P')

The undirected graph G :
S -> A -> B -> D
A -> S -> B -> C
B -> A -> S -> D -> F -> G
C -> A -> G -> E
D -> S -> B -> F
E -> C -> H
F -> B -> D -> J
G -> B -> J -> I -> H -> C
H -> E -> G -> T
I -> J -> G -> T
J -> F -> G -> I
T -> I -> H
M -> N -> O
N -> M
O -> K -> M
K -> O


The undirected graph G after running the BFS algorithm :
S -> A -> B -> D
A -> S -> C
B -> S -> F -> G
D -> S -> F
C -> A -> E
F -> B -> D -> J
G -> B -> J -> I -> H
E -> C
J -> F -> G
I -> G -> T
H -> G -> T
T -> H -> I


distance : {'S': 0, 'A': 1, 'B': 1, 'D': 1, 'C': 2, 'F': 2, 'G': 2, 'E': 3, 'H': 3, 'J': 3, 'I': 3, 'T': 4, 'M': inf, 'N': inf, 'O': inf, 'K': inf}

Attempting to run BFS on a vertex that does not exist in the graph: 
Vertex P not in graph!


# The Graph of the shortest paths from S to T in a undirected graph:

In [387]:
print('The undirected graph G :')
printG(G_notDirected)
print()

G,distance = path2vUndirected(G_notDirected,'S','T')
print('The graph of the shortest routes from S to T in a undirected graph:')
printG(G)
print('distance:',distance)

print()
print('Trying to find the shortest route between two vertices even though there are no routes between them: ')
path2vUndirected(G_directed,'K','B')

The undirected graph G :
S -> A -> B -> D
A -> S -> B -> C
B -> A -> S -> D -> F -> G
C -> A -> G -> E
D -> S -> B -> F
E -> C -> H
F -> B -> D -> J
G -> B -> J -> I -> H -> C
H -> E -> G -> T
I -> J -> G -> T
J -> F -> G -> I
T -> I -> H
M -> N -> O
N -> M
O -> K -> M
K -> O



distance:  {'S': 0, 'A': 1, 'B': 1, 'D': 1, 'C': 2, 'F': 2, 'G': 2, 'E': 3, 'H': 3, 'J': 3, 'I': 3, 'T': 4, 'M': inf, 'N': inf, 'O': inf, 'K': inf}
Direct the arcs from one level to the next:

S -> A -> B -> D
A -> C
B -> F -> G
C -> E
D -> F
F -> J
G -> J -> I -> H
H -> T
I -> T


!!!
distance:  {'S': 0, 'A': 1, 'B': 1, 'D': 1, 'C': 2, 'F': 2, 'G': 2, 'E': 3, 'J': 3, 'I': 3, 'H': 3, 'T': 4}
The directed graph G transpose after running the BFS algorithm from vertex T:
B -> S
G -> B
I -> G
H -> G
T -> H -> I


{'A': inf, 'S': 4, 'B': 3, 'D': inf, 'C': inf, 'F': inf, 'G': 2, 'E': inf, 'J': inf, 'I': 1, 'H': 1, 'T': 0}
Path from S to T in directed graph:
S -> B
B -> G
G -> I -> H
H -> T
I -> T


The graph of the s

# Finding the graph of the lightest routes:

In [385]:
print('The weighted graph:')
printG(G_weighted)
print('weight:',weight)
print()

d,p = DAG_Shortest_Path(G_weighted,'S',weight)
print('After using DAG - Shortest - Paths:\n')

G_weighted_DAG = makeGfromDis_wighted(d,G_weighted,weight)
printG(G_weighted_DAG)
print('Distance: ', d)

print()
print('Attempting to run DAG on a vertex that does not exist in the graph: ')
DAG_Shortest_Path(G_notDirected,'P',weight)

The weighted graph:
S -> D -> A -> L
A -> B
B -> G -> H
C -> T
D -> G
G -> C
H -> T
K -> T
L -> B
T


weight: {('S', 'A'): 0, ('S', 'L'): 0, ('L', 'B'): 1, ('S', 'D'): 10, ('A', 'B'): 1, ('B', 'G'): -3, ('B', 'H'): -1, ('C', 'T'): -9, ('D', 'G'): 3, ('G', 'C'): -4, ('H', 'T'): -15, ('K', 'T'): -3}

Order after topological sort:
['S', 'K', 'D', 'A', 'L', 'B', 'G', 'H', 'C', 'T']
Graph after initialize:
Distance: {'S': 0, 'A': inf, 'B': inf, 'C': inf, 'D': inf, 'G': inf, 'H': inf, 'K': inf, 'L': inf, 'T': inf}
After using DAG - Shortest - Paths:

S -> D -> A -> L
A -> B
B -> G -> H
C -> T
G -> C
H -> T
L -> B


Distance:  {'S': 0, 'A': 0, 'B': 1, 'C': -6, 'D': 10, 'G': -2, 'H': 0, 'K': inf, 'L': 0, 'T': -15}

Attempting to run DAG on a vertex that does not exist in the graph: 
Vertex P not in graph!


# The Graph of the lightest paths from S to T:

In [386]:
print('The weighted graph:')
printG(G_weighted)
print('weight:',weight)
print()

path,dis = FindPathBetween2V_wieghted(G_weighted,'S','T',weight)
print('The lightest path from S to T')
printG(path)
print('Distance: ', dis)

print()
print('Trying to find the lightest route between two vertices even though there are no routes between them: ')
FindPathBetween2V_wieghted(G_weighted,'K','B',weight)

The weighted graph:
S -> D -> A -> L
A -> B
B -> G -> H
C -> T
D -> G
G -> C
H -> T
K -> T
L -> B
T


weight: {('S', 'A'): 0, ('S', 'L'): 0, ('L', 'B'): 1, ('S', 'D'): 10, ('A', 'B'): 1, ('B', 'G'): -3, ('B', 'H'): -1, ('C', 'T'): -9, ('D', 'G'): 3, ('G', 'C'): -4, ('H', 'T'): -15, ('K', 'T'): -3}

Order after topological sort:
['S', 'K', 'D', 'A', 'L', 'B', 'G', 'H', 'C', 'T']
Graph after initialize:
Distance: {'S': 0, 'A': inf, 'B': inf, 'C': inf, 'D': inf, 'G': inf, 'H': inf, 'K': inf, 'L': inf, 'T': inf}
!!!!!
Graph G after running DAG - shortest - path from vertex S:
S -> D -> A -> L
A -> B
B -> G -> H
C -> T
G -> C
H -> T
L -> B


Distance:  {'S': 0, 'A': 0, 'B': 1, 'C': -6, 'D': 10, 'G': -2, 'H': 0, 'K': inf, 'L': 0, 'T': -15}
The graph G transpose: 
D -> S
A -> S
L -> S
B -> A -> L
G -> B
H -> B
T -> C -> H
C -> G


Weieght transpose:  {('A', 'S'): 0, ('L', 'S'): 0, ('B', 'L'): 1, ('D', 'S'): 10, ('B', 'A'): 1, ('G', 'B'): -3, ('H', 'B'): -1, ('T', 'C'): -9, ('G', 'D'): 3, ('C'